In [43]:
!kaggle datasets download -d sahilislam007/college-student-placement-factors-dataset


Dataset URL: https://www.kaggle.com/datasets/sahilislam007/college-student-placement-factors-dataset
License(s): MIT
college-student-placement-factors-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [44]:
import zipfile


nome_arquivo_zip = "college-student-placement-factors-dataset.zip"

diretorio_destino = "data_raw"

with zipfile.ZipFile(nome_arquivo_zip, 'r') as zip_ref:
    zip_ref.extractall(diretorio_destino)

print(f"Arquivos extraídos para a pasta '{diretorio_destino}'")

Arquivos extraídos para a pasta 'data_raw'


In [45]:
import findspark
import os


os.environ["SPARK_HOME"] = "C:\\spark\\spark-4.0.0-bin-hadoop3"
os.environ["JAVA_HOME"] = "C:\\Program Files\\Java\\jdk-17"

os.environ["PATH"] += os.pathsep + os.path.join(os.environ["SPARK_HOME"], "bin")

findspark.init()


from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("AvaliacaoSPARK").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True)
spark

In [46]:
df = spark.read.csv("data_raw/college_student_placement_dataset.csv", header=True, sep=",")
df.show(5)

+----------+---+---------------+----+--------------------+---------------------+----------------------+--------------------+------------------+---------+
|College_ID| IQ|Prev_Sem_Result|CGPA|Academic_Performance|Internship_Experience|Extra_Curricular_Score|Communication_Skills|Projects_Completed|Placement|
+----------+---+---------------+----+--------------------+---------------------+----------------------+--------------------+------------------+---------+
|   CLG0030|107|           6.61|6.28|                   8|                   No|                     8|                   8|                 4|       No|
|   CLG0061| 97|           5.52|5.37|                   8|                   No|                     7|                   8|                 0|       No|
|   CLG0036|109|           5.36|5.83|                   9|                   No|                     3|                   1|                 1|       No|
|   CLG0055|122|           5.47|5.75|                   6|                  

In [47]:
import faker
from faker import Faker
import random
from pyspark.sql.functions import col
import pandas as pd

In [48]:
df.columns

['College_ID',
 'IQ',
 'Prev_Sem_Result',
 'CGPA',
 'Academic_Performance',
 'Internship_Experience',
 'Extra_Curricular_Score',
 'Communication_Skills',
 'Projects_Completed',
 'Placement']

In [49]:
df.filter((col("IQ") < 70) | (col("IQ") > 160)).show()
print(f"Total de registros com IQ inválido: {df.filter((col('IQ') < 70) | (col('IQ') > 160)).count()}")

+----------+---+---------------+----+--------------------+---------------------+----------------------+--------------------+------------------+---------+
|College_ID| IQ|Prev_Sem_Result|CGPA|Academic_Performance|Internship_Experience|Extra_Curricular_Score|Communication_Skills|Projects_Completed|Placement|
+----------+---+---------------+----+--------------------+---------------------+----------------------+--------------------+------------------+---------+
|   CLG0034| 60|           7.44| 7.6|                  10|                  Yes|                     2|                   2|                 4|       No|
|   CLG0062| 69|           9.32|9.06|                   3|                   No|                     6|                  10|                 3|      Yes|
|   CLG0004| 51|           7.26|7.43|                   1|                   No|                     4|                   9|                 0|       No|
|   CLG0082| 68|           9.64|9.88|                   3|                  

In [50]:
df.filter(
    ((col("Academic_Performance") == 10) | (col("Academic_Performance") == 9)) & \
    (col("CGPA") < 6.0) & \
    (col("Prev_Sem_Result") < 6.0)
).show()
print(f"Total de registros inconsistentes: {df.filter(((col('Academic_Performance') == 10) | (col('Academic_Performance') == 9)) & (col('CGPA') < 6.0) & (col('Prev_Sem_Result') < 6.0)).count()}")

+----------+---+---------------+----+--------------------+---------------------+----------------------+--------------------+------------------+---------+
|College_ID| IQ|Prev_Sem_Result|CGPA|Academic_Performance|Internship_Experience|Extra_Curricular_Score|Communication_Skills|Projects_Completed|Placement|
+----------+---+---------------+----+--------------------+---------------------+----------------------+--------------------+------------------+---------+
|   CLG0036|109|           5.36|5.83|                   9|                   No|                     3|                   1|                 1|       No|
|   CLG0006| 80|            5.9|5.96|                  10|                  Yes|                     6|                   7|                 1|       No|
|   CLG0036| 98|           5.79|5.51|                   9|                   No|                    10|                   3|                 0|       No|
|   CLG0089|115|           5.56|5.14|                  10|                  

In [51]:
df_limpo = df.filter((col("IQ") >= 70) & (col("IQ") <= 160))

print(f"Número original de registros: {df.count()}")
print(f"Número de registros após remover IQ inválido: {df_limpo.count()}")

Número original de registros: 10000
Número de registros após remover IQ inválido: 9770


In [52]:
condicao_inconsistente = (
    ((col("Academic_Performance") == 10) | (col("Academic_Performance") == 9)) &
    (col("CGPA") < 6.0) &
    (col("Prev_Sem_Result") < 6.0)
)
df_final_limpo = df_limpo.filter(~condicao_inconsistente)

print(f"Número de registros antes de remover inconsistências: {df_limpo.count()}")
print(f"Número de registros após remover inconsistências: {df_final_limpo.count()}")

Número de registros antes de remover inconsistências: 9770
Número de registros após remover inconsistências: 9437


In [53]:
from pyspark.sql.functions import row_number,lit
from pyspark.sql.window import Window
w = Window().orderBy(lit('A'))
df_final_limpo = df_final_limpo.withColumn("id", row_number().over(w))
df_final_limpo.show(5)

+----------+---+---------------+----+--------------------+---------------------+----------------------+--------------------+------------------+---------+---+
|College_ID| IQ|Prev_Sem_Result|CGPA|Academic_Performance|Internship_Experience|Extra_Curricular_Score|Communication_Skills|Projects_Completed|Placement| id|
+----------+---+---------------+----+--------------------+---------------------+----------------------+--------------------+------------------+---------+---+
|   CLG0030|107|           6.61|6.28|                   8|                   No|                     8|                   8|                 4|       No|  1|
|   CLG0061| 97|           5.52|5.37|                   8|                   No|                     7|                   8|                 0|       No|  2|
|   CLG0055|122|           5.47|5.75|                   6|                  Yes|                     1|                   6|                 1|       No|  3|
|   CLG0004| 96|           7.91|7.69|               

In [54]:
fake = Faker('en_US')

num_registros = df_final_limpo.count()

# Gerar dados fictícios
data = []
for i in range(num_registros):
    primeiro_nome = fake.first_name()
    sobrenome = fake.last_name()
    cidade = fake.city()
    id=i+1
    data.append((primeiro_nome, sobrenome, cidade, id))

schema = ["primeiro_nome", "sobrenome", "cidade", "id"]

df_faker = pd.DataFrame(data, columns=schema)
df_faker.to_csv('data_raw/faker.csv', index=False)

In [55]:
df_nome_faker=spark.read.csv("data_raw\\faker.csv", header=True, sep=",")
df_nome_faker.show(5)


+-------------+---------+-------------+---+
|primeiro_nome|sobrenome|       cidade| id|
+-------------+---------+-------------+---+
|     Margaret|    Lewis|     Troyfurt|  1|
|       Robert|     Cain|  Lake Robert|  2|
|        Jerry| Santiago|Underwoodview|  3|
|     Nicholas|    Drake|   Vincentton|  4|
|     Lawrence|   Hodges|  New Shannon|  5|
+-------------+---------+-------------+---+
only showing top 5 rows


In [56]:
df_joined=df_nome_faker.join(df_final_limpo,on='id', how='inner')
df_joined.show(5)

+---+-------------+---------+-------------+----------+---+---------------+----+--------------------+---------------------+----------------------+--------------------+------------------+---------+
| id|primeiro_nome|sobrenome|       cidade|College_ID| IQ|Prev_Sem_Result|CGPA|Academic_Performance|Internship_Experience|Extra_Curricular_Score|Communication_Skills|Projects_Completed|Placement|
+---+-------------+---------+-------------+----------+---+---------------+----+--------------------+---------------------+----------------------+--------------------+------------------+---------+
|  1|     Margaret|    Lewis|     Troyfurt|   CLG0030|107|           6.61|6.28|                   8|                   No|                     8|                   8|                 4|       No|
|  2|       Robert|     Cain|  Lake Robert|   CLG0061| 97|           5.52|5.37|                   8|                   No|                     7|                   8|                 0|       No|
|  3|        Jerry| 